In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd
import mne

#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
print('seaborn ver = ', sns.__version__)
#assert sns.__version__ == '0.12.2', sns.__version__
import matplotlib as mpl
COLOR = 'black'
COLOR_INV = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams["figure.facecolor"] = COLOR_INV
mpl.rcParams["axes.facecolor"] = COLOR_INV


sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
           'grid.color':'grey', 'grid.alpha':0.5})

tgtc = ['b','g','r','brown']


from glob import glob
from pathlib import Path
dmeg = '/home/demitau/data_Quentin/pilot_studies/context_change_MEG/'
fnbs = []
subjects = []
subj2path = {}
subj2dsnames = {}
for fnf in os.scandir(dmeg):
    if fnf.is_dir():
        print(fnf, fnf.name)
        subjects += [fnf.name]
        subj2path[fnf.name] = fnf.path
        
        subj2dsnames[fnf.name] = []
        
        for fnf2 in os.scandir(fnf.path):
            #print(fnf2.name)
            if fnf2.name.endswith('.ds'):
                subj2dsnames[fnf.name] += [fnf2.path]
        subj2dsnames[fnf.name] = list(sorted(subj2dsnames[fnf.name]))
    
display(list(sorted(subj2path)))

subj = 'pilot_Dmitrii_20230426'
print('subj ', subj)  

subj_nodate = '_'.join( subj.split('_')[:2] )
#fnbs = pjoin( subj2path[subj] , 'behav', subj_nodate)
#d = pjoin( subj2path[subj] , 'behav', subj_nodate)
path_behav = pjoin( subj2path[subj] , 'behav')
fnbs = []
for fnf in glob(pjoin(path_behav,'*_trigger.log') ):
    name = Path(fnf).name
    fnb = name.replace('_trigger.log','')
    fnbs += [fnb]
    #print( f'"{fnb}",' )

print(fnbs)

# TODO: read edf file

In [ ]:
from behav_proc import readParamFiles, addBasicInfo, getGeomInfo


dfs = []
for fnb in fnbs:
    fnp = fnb + '.param'
    params, phase2trigger, trigger2phase, CONTEXT_TRIGGER_DICT  = readParamFiles(fnp, path_behav)
    
    fn = fnb + '.log'
    fnp = fnb + '.param'

    with open(pjoin(path_behav,fn), 'r') as f:
         l = f.readline()
         truelen = len( l.split(',') )

    r = ('trial_index, current_phase_trigger, tgti_to_show,'
       ' vis_feedback_type, trial_type, special_block_type, block_ind, '
        ' feedbackX, feedbackY, unpert_feedbackX, unpert_feedbackY,'
         ' error_distance, target_coordX, target_coordY, '
         'feedbackX_when_crossing, feedbackY_when_crossing, '
         'jax1, jax2, reward, time, time_abs')
    r = r.replace(' ','')
    colnames = r.split(',')
    print('len(colnames) = ',len(colnames),'truelen = ',truelen,colnames)
    assert truelen == len(colnames)

    nbad = 3  # recompense strip
    #colnames = ['trial_index', 'current_phase_']
    df = pd.read_csv(pjoin(path_behav,fn), skipfooter = nbad, on_bad_lines='warn', header=0,
                    names=colnames, encoding='latin1')
    #encoding='latin-1'
    subj_ = fnb.split('_')[0]    
    df['subject'] = subj_
    dfs += [df]
    
    df['phase'] = df.apply(lambda row: trigger2phase[row['current_phase_trigger']], 1)
    
    fnf = pjoin(path_behav, fnb + '_trigger.log')
    from behav_proc import loadTriggerLog
    dftriglog = loadTriggerLog(fnf,CONTEXT_TRIGGER_DICT)

print( 'len = {}, maxtime in min = {}'.format( len(df), df['time'].max() / 60 ) )

In [ ]:
def mplresestcol():
    COLOR = 'black'
    COLOR_INV = 'white'
    mpl.rcParams['text.color'] = COLOR
    mpl.rcParams['axes.labelcolor'] = COLOR
    mpl.rcParams['xtick.color'] = COLOR
    mpl.rcParams['ytick.color'] = COLOR
    mpl.rcParams["figure.facecolor"] = COLOR_INV
    mpl.rcParams["axes.facecolor"] = COLOR_INV

# Read meg

In [ ]:
#raw.pick_types

In [ ]:
raws0 = []
for fnf_ds in subj2dsnames[subj]:
    print(fnf_ds)

#raw_fname1 = data_path / 'MEG' / subject / 'S01_AEF_20131218_01.ds'
#raw_fname2 = data_path / 'MEG' / subject / 'S01_AEF_20131218_02.ds'
#erm_fname = data_path / 'MEG' / subject / 'S01_Noise_20131218_01.ds'
# In the memory saving mode we use preload=False and use the memory efficient IO which loads the data on demand. However, filtering and some other functions require the data to be preloaded into memory.

    raw = mne.io.read_raw_ctf(fnf_ds)
    print(raw.n_times, raw.times[-1], raw.times[-1]/60 )
    raws0 += [raw]
    
    # if I drop ref meg it writes "". So maybe I won't for now
    #Removing 5 compensators from info because not all compensation channels were picked.
    #raw = raw.drop_channels(chns_ref_meg)    

# Here we ignore that these have different device<->head transforms
raw = mne.io.concatenate_raws(raws0, on_mismatch='ignore')
#raw_erm = read_raw_ctf(erm_fname)
m = {'SCLK01-177':'syst',
 'EEG063-2800':'emg',
 'EEG064-2800':'emg',
 'UADC001-2800':'misc', 
 'UADC010-2800':'eog',
 'UADC011-2800':'eog',
 'UADC012-2800':'misc',
 'UPPT001':'stim',
 'UPPT002':'syst'}

# The EEG063 and 64 are EMG
# UPPT002 is the parralel port from the interface box (only registers button presses)
# UADC10, 11 and 12 are eyelink for x, y position and pupil size

chn_diode = 'UADC001-2800'
chn_pupil_size = 'UADC012-2800'
chn_time = 'SCLK01-177'

to_drop =['EEG063-2800']
raw = raw.set_channel_types(m)

chn2cht = dict( zip( raw.ch_names, raw.get_channel_types() ) )
chns_nontriv = [chn for chn,cht in chn2cht.items() if cht not in ['mag', 'ref_meg']]
chns_ref_meg = [chn for chn,cht in chn2cht.items() if cht in ['ref_meg']]
#raw_nontriv.set_channel_types(m)

#dat = raw_nontriv.get_data('stim')[0]

events = mne.find_events(raw, shortest_event=2)
dfrest = df.query('phase == "REST"')
restmintime = dfrest['time_abs'].min()
#df['time_MEG_aligned'] = df['time_abs'] + (restmintime_meg - restmintime)
df['time_since_first_REST'] = df['time_abs']  - restmintime

print(restmintime)
from meg_proc import events2df
dat_time ,times = raw[chn_time] 
dfev =events2df(events, raw.info, dat_time, trigger2phase, CONTEXT_TRIGGER_DICT, restmintime)

import gc; gc.collect()

### debug

#### align time

In [ ]:
# this is not accurate, does not take into account script restarts
restmintime_meg2 = dfev.query('phase == "REST"')['time'].min()
dfev['time_since_first_REST2'] = dfev['time'] - restmintime_meg2
mvt_starts_megtrig2 = dfev.query('phase == "TARGET_AND_FEEDBACK"')
mvt_starts_megtrig2_REST = mvt_starts_megtrig2['time_since_first_REST2']

In [ ]:
#dfev['time_since_first_REST'] = dfev['time'] - restmintime_meg

In [ ]:
mvt_starts = df.query('phase == "TARGET_AND_FEEDBACK"').groupby('trial_index').min('time')
mvt_starts_megtrig = dfev.query('phase == "TARGET_AND_FEEDBACK"')

tcoln = 'time_since_first_REST'
#mvt_starts_megtrig[tcoln].reset_index(), mvt_starts[tcoln].reset_index()
dif = mvt_starts_megtrig[tcoln].values - mvt_starts[tcoln].values
print('dff in sec = ', np.max(np.abs( dif )) )

In [ ]:
mplresestcol()

In [ ]:
dif = mvt_starts_megtrig2_REST - mvt_starts[tcoln].values
plt.plot(mvt_starts_megtrig[tcoln], dif)
plt.xlabel('[sec]')
plt.ylabel('difference in time measurement, [sec]')
plt.title(f'Times_MEG (event sample) - Times_log')

In [ ]:
plt.plot(mvt_starts_megtrig[tcoln], dif)
plt.xlabel('[sec]')
plt.ylabel('difference in time measurement, [sec]')
plt.title(f'Times_MEG ({chn_time}) - Times_log')

#### general 

In [ ]:
raw

In [ ]:
# use ref meg to denoise
# https://mne.tools/stable/auto_examples/preprocessing/find_ref_artifacts.html

In [ ]:
chns_ref_meg0 = np.array(raw.ch_names)[mne.pick_types(raw.info, meg=False, ref_meg=True)]
set(chns_ref_meg) == set(chns_ref_meg0)

chn2cht

raw_nontriv = raw.copy()
raw_nontriv.pick_channels(chns_nontriv)

display( list(zip(chns_nontriv, [chn2cht[chn] for chn in chns_nontriv] ) ))

In [ ]:
%matplotlib inline
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams["figure.facecolor"] = COLOR_INV
mpl.rcParams["axes.facecolor"] = COLOR_INV


In [ ]:
# ['SCLK01-177':'misc'
#  'EEG063-2800':''
#  'EEG064-2800',
#  'UADC001-2800':'diode', 
#  'UADC010-2800':'EOG',
#  'UADC011-2800':'eyelink',
#  'UADC012-2800':'eyelink?',
#  'UPPT001':'trigger',
#  'UPPT002':'?']

# UPPT002 mostly zero, only 682 equals to 2, never to 1. 
#      Only changes to 2 in the beginning for a short time (682 samples =0.568333 s)
# EEG063-2800  constant 
# EEG064-2800  not exactly constant  but very small. Changes on the order of minutes
# very similar to times but with jumps around rec breaks, also it starts not from 0. 
#     Maybe time in seconds since machine start in the morning of rec day

In [ ]:
# ecg, eeg, emg, eog, exci, ias, misc, resp, seeg, dbs, stim, syst,
#    ecog, hbo, hbr, fnirs_cw_amplitude, fnirs_fd_ac_amplitude,
#    fnirs_fd_phase, fnirs_od, temperature, gsr

In [ ]:
for chn in chns_nontriv:
    d,times = raw_nontriv[chn]

    print(chn)
    print( 'minmax std = ',np.min(d), np.max(d), np.std(d) )
    inds = np.where( np.abs(d[0]) >0)[0]
    if len(inds):
        print( 'first, len =', d[0,inds[0]], len(inds) )

In [ ]:
%matplotlib qt
chn = 'UPPT001'
plt.figure(20)
inds = raw_nontriv.time_as_index([0,4])

d,times = raw_nontriv[chn, inds[0]:inds[1]]
plt.plot(times,d[0])
#plt.xlim(0,3)

In [ ]:
with pd.option_context('display.max_rows', None):
    display(dfev[:50])

In [ ]:
import gc; gc.collect()

In [ ]:
raw.plot?

In [ ]:
raw

In [ ]:
#raw2 = raw.pick_types('mag')
#raw2.ch_names
#set(raw.ch_names) - set(raw2.ch_names)
#set([chn[:5] for chn in raw.ch_names])

In [ ]:
%matplotlib qt
raw.plot(theme='dark', clipping=3, duration=30, n_channels=50,
        events=events);

In [ ]:
anns = mne.preprocessing.annotate_amplitude(raw, picks='meg')
anns

In [ ]:
from autoreject import AutoReject
ar = AutoReject()
epochs_clean = ar.fit_transform(epochs)  

In [ ]:
from autoreject import get_rejection_threshold
reject = get_rejection_threshold(epochs)  

In [ ]:
from autoreject import Ransac
rsc = Ransac()
epochs_clean = rsc.fit_transform(epochs)  

# Raw artif

In [ ]:
# Can I allow myself to reject entire epochs if I care about continuous history?
# if not then I have to repair raw perhaps. Or impute data I don't like

In [ ]:
raw.load_data()

In [ ]:
n_jobs = 40
# The threshold is data dependent, check the optimal threshold by plotting
# ``scores_muscle``.
#threshold_muscle = 5  # z-score
#<Annotations | 119 segments: BAD_muscle (119)>
#Tot dur =  114.12666666666709 of total  2170.679166666667

threshold_muscle = 6  # z-score
# <Annotations | 106 segments: BAD_muscle (106)>
# Tot dur =  88.23583333333399 of total  2170.679166666667

threshold_muscle = 7  # z-score
# Tot dur =  1.4683333333333621 of total  2170.679166666667

from mne.preprocessing import annotate_muscle_zscore
# Choose one channel type, if there are axial gradiometers and magnetometers,
# select magnetometers as they are more sensitive to muscle activity.
annot_muscle, scores_muscle = annotate_muscle_zscore(
    raw, ch_type="mag", threshold=threshold_muscle, min_length_good=0.2,
    filter_freq=[110, 140], n_jobs=n_jobs)

annot_muscle.save( pjoin(subj2path[subj], 'preproc', f'muscle_thr{threshold_muscle}-ann.txt' ) )

# Sudden break observe

In [ ]:
tuple( raw.time_as_index([620,660]) )

In [ ]:
np.arange(*ts)

In [ ]:
ts = raw.time_as_index([620,660] )
r = raw.get_data('MLC12-2805',tmin=620,tmax=660, return_times=True)[0]

In [ ]:
len(raw.times)

In [ ]:
raw.times[-1]

In [ ]:
tt  = 0.66
ts = raw.time_as_index([645.6 + tt ,647 - 0.065 -tt] )
dat,times = raw['MLC12-2805',slice(*ts)]
plt.plot(times,dat[0])
plt.show()

In [ ]:
ts = raw.time_as_index([0,4200] )
dat,times = raw[rawmeg.ch_names[:3] + ['UPPT001'],slice(*ts)]
for i in range(len(dat)):
    plt.plot(times,dat[i])
    plt.show()

In [ ]:
#ts = raw.time_as_index([750,1300] )
ts = raw.time_as_index([300,1200] )
dat,times = raw[rawmeg.ch_names[:3] + ['UPPT001'],slice(*ts)]
for i in range(len(dat)):
    plt.plot(times,dat[i])
    plt.show()

In [ ]:
#ts = raw.time_as_index([750,1300] )
ts = raw.time_as_index([1000,1200] )
dat,times = raw[['MLC12-2805','MLC14-2805', 'MRO24-2805'],slice(*ts)]
for i in range(len(dat)):
    plt.plot(times,dat[i])
    plt.show()

In [ ]:
raw.plot?

In [ ]:
rawmeg = raw.copy()
rawmeg.pick_types(meg=True)

In [ ]:
#rawmeg.plot(butterfly=1,duration=50,events=events);
#group_by='position', 
rawmeg.plot(start=950, duration=300, butterfly=1, events=events, 
            scalings= 'auto');

In [ ]:
%matplotlib inline
#rawmeg.plot(butterfly=1,duration=50,events=events);
#group_by='position', events=events, 
rawmeg.plot(start=950, duration=300, butterfly=1, 
            scalings= 'auto');

In [ ]:
rawmeg.plot?

In [ ]:
4+3

In [ ]:
# 620-660 emptiness

In [ ]:
print(annot_muscle)
print('Tot dur = ',np.sum(annot_muscle.duration), 'of total ',raw.times[-1] )

In [ ]:
gc.collect()

In [ ]:
raw.set_annotations(annot_muscle);

In [ ]:
raw.plot(start = 950, duration=300, group_by='type', scalings= 'auto',
        butterfly=1, events=events);

In [ ]:
phase2trigger

# Epochs

In [ ]:
y = np.roll( np.arange(10), 1, axis=0)
#y = #np.roll(x,1,axis=1)
y[0] = -100000
y

In [ ]:
#CONTEXT_TRIGGER_DICT

In [ ]:
#mne.Epochs?

In [ ]:
# multiclass CSP classif
# https://mne.discourse.group/t/csp-for-multi-classification/1749/3

In [ ]:
#ITI_duration, motor_prep_duration, time_feedback, time_at_home

In [ ]:
ITI_duration = float(params["ITI_duration"])
motor_prep_duration = float(params["motor_prep_duration"])
time_feedback = float(params["time_feedback"])
time_at_home = float(params["time_at_home"])
perf_feedback_duration = float(params["perf_feedback_duration"])
perf_notif_start_delay = float(params["perf_notif_start_delay"])

print("ITI_duration:", ITI_duration)
print("motor_prep_duration:", motor_prep_duration)
print("time_feedback:", time_feedback)
print("time_at_home:", time_at_home)
print("perf_feedback_duration:", perf_feedback_duration)
print("perf_notif_start_delay:", perf_notif_start_delay)
# REST, GOCUE, MVT, ITI

In [ ]:
params['perf_notif_start_delay']
params['perf_feedback_duration']

In [ ]:
#-2,5

In [ ]:
coln_error_area = 'error_area2_signed_nn_scaled_ed'
coln_error_lh = 'error_lh_ang'
coln_error_ep = 'error_endpoint_ang'

offset = 0.3
coln_tgt = 'tgti_to_show'
varnames = [coln_tgt, coln_error_ep, coln_error_area, 'prev_' + coln_error_area,
               coln_error_lh, 'prev_' + coln_error_lh]
# coln_error_ep? reward~coln_error_area?
vartypes = {coln_tgt:'cat', coln_error_area:'cont', 'prev_' + coln_error_area:'cont', 
            coln_error_lh:'cont', 'prev_'+coln_error_lh:'cont',
            coln_error_ep:'cont',  'reward_blink_color':'cat'}
# rel to ITI is error on this trial or prev? I think it is this trial and we shift to next only when we go to REST
phase2epochs_gen_info = {
    'ITI':
         {'tlim0':[-motor_prep_duration-time_feedback,ITI_duration], 
          'tlims':[(-0.5,0), (0,0.5)], 
           'varnames':varnames }, 
    'GO_CUE_WAIT_AND_SHOW':
         {'tlim0':[0,motor_prep_duration+time_feedback + offset],
          'tlims':[(-0.5,0), (0,0.5)], 
           'varnames':varnames },
     'TARGET_AND_FEEDBACK':
         {'tlim0':[-motor_prep_duration,time_feedback + offset],
          'tlims':[(-0.5,0), (0,0.5)], 
           'varnames':varnames} }

for desired_phase in phase2epochs_gen_info:
    epginfo = phase2epochs_gen_info[desired_phase]
    epginfo['tlims'] = dict(list(zip(['prestart','poststart'], epginfo['tlims'])) )

In [ ]:
phase2epochs_gen_info

In [ ]:
gc.collect()

In [ ]:
phase2tlimn2ev = phase2ev2tlimn

In [ ]:
# should be ran after prep Y
p = pjoin(subj2path[subj], 'preproc')

tlim_test = None
#tlim_test = -0.5,0.5

load=False
phase2tlimn2ep = {}
phase2tlimn2ev = {}

#decim = 6
decim = 2

#reject = dict(mag=4e-12, eog=250e-6)
#reject = dict(mag=4e-12)  # lead to rejection of 4 TARGET_AND_FEEDBACK epochs
reject = None

p0 = pjoin( subj2path[subj], 'preproc')
if not os.path.exists(p0):
    if load:
        raise ValueError('dir not exists')
    os.makedirs(p0)

for desired_phase in ['TARGET_AND_FEEDBACK', 'GO_CUE_WAIT_AND_SHOW', 'ITI' ]:
#for desired_phase in ['TARGET_AND_FEEDBACK']:
    phase2tlimn2ep[desired_phase] = {}
    phase2tlimn2ev[desired_phase] = {}
    epginfo = phase2epochs_gen_info[desired_phase]
    if tlim_test is not None:
        print('!!! Using tlim_test = ',tlim_test)
        tlims = {'test':tlim_test}
    else:
        tlims = epginfo['tlims']
    for tlim_name, tlim in tlims.items():
        tmin,tmax = tlim
        trigs_desired_phase = [trig for trig, tpl in CONTEXT_TRIGGER_DICT.items() if tpl[-1] == desired_phase]
        print(desired_phase,tlim_name, tmin,tmax)

        #continue # to check that we load right things
        dfev_mxph = phase2dat[desired_phase]['dfev_mxph']
        events_cur = events[dfev_mxph.index.values] # I hope it is correct

        assert set( events_cur[:,2] ) == set(trigs_desired_phase), (set( events_cur[:,2] ), set(trigs_desired_phase))

        fnfep = pjoin(p,f'{desired_phase}_{tlim_name}_{tmin:.2f},{tmax:.2f}_decim{decim}-epo.fif')
        fnfev = pjoin(p,f'{desired_phase}_{tlim_name}_{tmin:.2f},{tmax:.2f}_decim{decim}-ave.fif')
        if load:
            epochs = mne.read_epochs(fnfep)
            evoked = mne.read_evokeds(fnfev)
        else:
            # these values are from 
            # https://mne.tools/stable/auto_tutorials/io/60_ctf_bst_auditory.html#sphx-glr-auto-tutorials-io-60-ctf-bst-auditory-py
            epochs = mne.Epochs(raw, events_cur, event_id=trigs_desired_phase, 
                                tmin=tmin, tmax=tmax, picks = 'meg', reject = reject,
                                baseline=(None, None), preload=1, decim = decim)
            epochs.save(fnfep, overwrite=1)
            evoked = epochs.average()  # slow
            evoked.save(fnfev, overwrite=1)

        #phase2ep[desired_phase] = epochs
        #phase2ev[desired_phase] = evoked
        phase2tlimn2ep[desired_phase][tlim_name] = epochs
        phase2tlimn2ev[desired_phase][tlim_name] = evoked
        
    import gc; gc.collect()

In [ ]:
import gc; gc.collect()

### autoreject

In [ ]:
autoreject.AutoReject?

In [ ]:
epochs

In [ ]:
# pretty long, tries to find a good threshold
import autoreject
ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11,
                           n_jobs=50, verbose=True, picks='meg')
# one could fit on epochs[:N] to save time
# fewer epochs for fit makes reject 25% more epochs, misses 2
ar.fit(epochs)  # fit on a few epochs to save time

epochs_ar, reject_log = ar.transform(epochs, return_log=True)
reject_log.plot('horizontal'); # based on 40

np.savez(pjoin(p,f'AR_{desired_phase}_{tmin:.2f},{tmax:.2f}_decim{decim}-thrs.npz'), ar.threshes_ )

fnfep = pjoin(p,f'AR_{desired_phase}_{tmin:.2f},{tmax:.2f}_decim{decim}-epo.fif')
epochs_ar.save(fnfep)

# for TARGET_AND_FEEDBACK
#ar.consensus_ == 0.2
# Dropped 14 epochs: 19, 34, 37, 42, 62, 79, 181, 190, 260, 303, 304, 305, 306, 377

In [ ]:
evoked

In [ ]:
evoked.plot?

In [ ]:
axi

## plot

In [ ]:
nr=3;nc=1
ww = 10; hh=3
#fig,axs = plt.subplots(nr,nc,figsize=(nc*ww,nr*hh))
#axs = axs.flatten()
#%matplotlib inline
phases = ['ITI', 'GO_CUE_WAIT_AND_SHOW', 'TARGET_AND_FEEDBACK']
for axi,desired_phase in enumerate(phases):
    epginfo = phase2epochs_gen_info[desired_phase]
    tmin,tmax = epginfo['tlim']
    #ax = axs[axi]
    fnfep = pjoin(p,f'{desired_phase}_{tmin:.2f},{tmax:.2f}_decim{decim}-ave.fif')
    if not os.path.exists(fnfep):
        print('not exits ',fnfep)
        continue
    print(desired_phase,  tmin,tmax)
    evoked = mne.read_evokeds(fnfep)[0]

    #, axes=ax
    fig = evoked.plot(time_unit = 's', window_title=desired_phase);
    
    #fig.suptitle(desired_phase)

# Decoding

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
#from mne.decoding.csp import CSP
#csp = CSP(n_components=n_classes, reg=None, log=True, norm_trace=False)
#### for CSP
#log_reg = LogisticRegression()
#clf = Pipeline([('CSP', csp), ('Reg', log_reg)])

### prep Y

In [ ]:
import sys, traceback
from behav_proc import getGeomInfo, addBasicInfo
home_position, target_coords = getGeomInfo(params)
hitr = float(params['radius_target']) + float(params['radius_cursor']) / 2

dfcs = []; dfccs = []; dfcpcs = []; dfctcs = [];    
for df in dfs:
    print('---------------  Starting processing new df')                    
    try:
        df, dfc, dfcc, dfcpc, dfctc = addBasicInfo(df, phase2trigger, params,
                                         home_position, target_coords, training_end_sep_trial=True)
        from behav_proc import printPretrialMistakesDiag
        printPretrialMistakesDiag(df, params)  
    except Exception as e:
        print('EXC',e)
        exc_info = sys.exc_info()
        exc = traceback.TracebackException(*exc_info, capture_locals=True)
        ei = exc_info[2]

        #display(ei.tb_frame)
        psf = ei

        lfprev = None; lf = None
        lfs = []
        while psf.tb_frame is not None:
            stackframe = psf.tb_frame
            display(stackframe)
            lfprev = lf
            lf = stackframe.f_locals
            lfs += [lf]
            psf = psf.tb_next    
            if psf is None:
                break    

In [ ]:
coln_error = 'error_area2_signed_nn_scaled_ed'
dfcc[f'prev_{coln_error}'] =  dfcc[coln_error].shift()
coln_error = 'error_lh_ang'
dfcc[f'prev_{coln_error}'] =  dfcc[coln_error].shift()

In [ ]:
from behav_proc import compareTriggers
compareTriggers(df,dfev,dftriglog)

In [ ]:
from meg_proc import checkSeqConsist
r,badis_dfev, badis_df, good_inds_dfev, good_inds_df = checkSeqConsist(df,dfev)

#### debug

In [ ]:
dfev.query('phase.isna()')['trigger'].unique()

In [ ]:


 #return r, bad_inds_dfev, bad_inds_df, good_inds_dfev, good_inds_df                
assert (dfev.loc[badis_dfev.values,'trial_index'].values ==  df.loc[badis_df.values,'trial_index'].values).all()

display(dfev.loc[badis_dfev.values].groupby('phase').size())
display(dfev.loc[badis_dfev.values[:10]])
#badis

# we want correspondance of trial_indices for bad as well

In [ ]:
display(dfev.loc[good_inds_dfev.values[:10],cols])

In [ ]:
dfev_mxph = dfev_mx_good.query('phase == @ph')

In [ ]:
#dfev_mxph['phase']

In [ ]:
from behav_proc import alignDfs
#cols = ['trial_index','vis_feedback_type','tgti_to_show','trial_type']
#dfmergealign = alignDfs(dfy,cols, dfcc,cols, '_cc', verbose=1)
cols = ['trial_index','vis_feedback_type','tgti_to_show','phase']
dfmergealign = alignDfs(dfy,cols, dfev_mxph,cols, '_ev', verbose=1)
dfmergealign.iloc[:14]

In [ ]:
ph

In [ ]:
dfmergealign = alignDfs(dfcc,cols, dfev_mxph,cols, '_ev', verbose=1)
dfmergealign.iloc[:14]

In [ ]:
dfy[cols + ['trial_type', 'trial_type']]

In [ ]:
dfev_mxph[cols]

In [ ]:
#df.compare(dfy[cols].re, dfev_mxph[cols])
# df.align

In [ ]:
dfccr = alignDfs(dfcc, cols, dfcc2, cols)
dfccr.iloc[:10]

In [ ]:
#def aggRows(df):
df

In [ ]:
grp_perti.agg?

In [ ]:
df[['trial_index','phase']].iloc[:50]

In [ ]:
dfc.columns

In [ ]:
#grp['time_since_first_REST'].min()

In [ ]:
grp.idxmin?

In [ ]:
len(dfcc2)

In [ ]:
len(dfcc)

In [ ]:
grp = dfc.groupby(['trial_index'])    
idx = grp['time_since_first_REST'].idxmin()
dfcc2 = dfc.loc[idx]
#idx = grp['time'].transform(max) == dfc['time'] #.size()      
#grp.                                                                                                                                              
#dfcc = dfc.loc[idx]                                                                                                                                
# it DOES NOT include pauses                                                                                                                       
#dfcc = dfcc.reset_ind

In [ ]:
#dfev.loc[good_inds_dfev]

In [ ]:
#(dfev['phase'] == 'TRAINING_START').size

In [ ]:
# check inconsistencies if found
cols = ['trial_index', 'trial_index_log','phase',
            'phase_log','tgti_to_show','tgti_to_show_log',
            'vis_feedback_type','vis_feedback_type_log',
           'trial_type','trial_type_log',
           'block_ind_log']
i0 = diftgt[0]
i = i0 -4; print(i0)
display(df_corresp_test[cols].loc[i:i+7])
display(dftmpmerge[cols].loc[i:i+7])

#### end debug

In [ ]:
len(dfev), len(good_inds_dfev)

In [ ]:
from behav_proc import aggRows
grp = dfev.groupby(['trial_index', 'phase'])
dfev_mx = aggRows(dfev, 'time_since_first_REST', 'max', grp)

dfev_good = dfev.loc[good_inds_dfev]
grp_good = dfev_good.groupby(['trial_index', 'phase'])
dfev_mx_good = aggRows(dfev_good, 'time_since_first_REST', 'max', grp_good)

print(len(dfev_mx), len(dfev_mx_good))

In [ ]:
len(epochs)

In [ ]:
len(dfev_mxph), np.max(dfev_mxph.index)

In [ ]:
phase2best_neg_csr = {'REST':-3, 'GO_CUE_WAIT_AND_SHOW':-2, 'TARGET_AND_FEEDBACK':-1, 'ITI':0 }
phase2dat = {}
for ph in phase2best_neg_csr:
    phase2dat[ph] = {}
#for ph in phase2epochs_gen_info:
    print(ph)
    csr_neg = phase2best_neg_csr[ph]
    #dfy = df.query('phase == @ph and subphase_relation == "first" and csr_neg_within_trial == @csr_neg')
    dfy = df.loc[good_inds_df].query(
        'phase == @ph and subphase_relation == "last" and csr_neg_within_trial == @csr_neg')
    

    #if ph in ['REST',]
    #dfev_mxph = dfev_mx.query('phase == @ph')
    dfev_mxph = dfev_mx_good.query('phase == @ph')
    
    print(len(dfev_mxph), len(dfy), len(dfcc) )
    
    vs1 = dfev_mxph[['vis_feedback_type','tgti_to_show']].values
    tis = dfev_mxph['trial_index']
    dfcc_ = dfcc.query('trial_index in @tis')
    
    #vs1 = dfev.query('phase == @ph')[['vis_feedback_type','tgti_to_show']].values
    vs2 = dfy[['vis_feedback_type','tgti_to_show']].values
    # remember that we defined dfcc from dfc which is only for one phase
    vs3 = dfcc_[['vis_feedback_type','tgti_to_show']].values
    print('   ',len(vs1))
    try:        
        assert len(vs1) == len(vs2), f'diff lengths ({len(vs1),len(vs2)})'        
        assert np.all( vs1[:,0] == vs2[:,0] ) and np.all( vs1[:,1] == vs2[:,1] )
        assert np.all( vs1[:,0] == vs3[:,0] ) and np.all( vs1[:,1] == vs3[:,1] )
    except AssertionError as e:
        print('  err', e)
        print(np.where( vs1[:,0] != vs2[:,0] )[0], np.where( vs1[:,1] != vs2[:,1] )[0])
        raise e
        
    phase2dat[ph]['dfev_mxph'] = dfev_mxph  # has indices consistent with original event thus with epochs
    if ph not in phase2epochs_gen_info:
        continue
    varnames_cur = phase2epochs_gen_info[ph]['varnames']
    varname2y = {}
    for vn in varnames_cur:
        y = dfcc_[vn].to_numpy()
        varname2y[vn] = y
        Y = y[:,None]
    phase2dat[ph]['varname2y'] = varname2y

In [ ]:
dfev_mxph.index.max()

In [ ]:
len(events)

In [ ]:
#phase2epochs_gen_info

In [ ]:
sl = slice(20)
for sl in [slice(20), slice(-10,None)]:
    print('---')
    for vs in [dfcc['trial_index'].values, dfev_mxph['csr_REST'].values ]:
        print(len(vs), vs[sl])

In [ ]:
import gc; gc.collect()

### debug getting y

In [ ]:
grp_perti = df.groupby('trial_index')
df_trst = grp_perti.agg(agg_d)

In [ ]:
# where we have strange numbers of phases?
dfsz = df.groupby(['trial_index','phase']).size().reset_index()
dfsz2 = dfsz.groupby('trial_index').size()
inds_sm = dfsz2[dfsz2 < 4].index.to_numpy()
print( inds_sm )
print( dfsz2[dfsz2 > 4].index.to_numpy() )

# TRAINING_END (one phase), Pause (two phases), break (two phases) is separate trial
display (dfsz.query( 'trial_index in @inds_sm' ))

In [ ]:
# i =1505
# dftmp.loc[i-3:i+3, ['phase','time'] ]

# with pd.option_context('display.max_rows', 500):
#     display(dftmp.loc[::10, ['phase','time']])

In [ ]:
vslog = phase2df['REST']['time_since_first_REST'].values
vsev = phase2dfev['REST']['time_since_first_REST'].values

for i in range(min(len(vslog),len(vsev))):
    d = vslog[i] - vsev[i]
    print(f'vslog = {vslog[i]:.3f}, vsev= {vsev[i]:.3f}, d={d:.6f} ')

plt.figure(figsize=(29,3))
ax = plt.gca()
ax.vlines( phase2df['REST']['time_since_first_REST'] , 0,1, color='b')
df_ = dfev.query('phase == "REST"')
ax.vlines( df_['time_since_first_REST'] , 0,1, color='r', ls=':')

In [ ]:
coln_error = 'error_area2_signed_nn_scaled_ed'
dfcc[coln_error]

In [ ]:
dfrest = df.query('phase == "REST"')
restmintime = dfrest['time_abs'].min()
print(restmintime)

restmintime_meg = dfev.query('phase == "REST"')['time_megchan'].min()
print(restmintime_meg)

In [ ]:
#a

## Run

In [ ]:
np.sum( np.isinf(y) ), np.sum( np.isnan(y) )

In [ ]:
from mne.decoding import SlidingEstimator, LinearModel

In [ ]:
LinearModel?

In [ ]:
import mne
mne.decoding.__file__

In [ ]:
from scipy.stats import spearmanr
from csp_my import SPoC
#from mne.decoding.csp import SPoC as SPoC_orig
from jr.gat import scorer_spearman
from mne.decoding import cross_val_multiscore

In [ ]:
#rom sklearn.utils import parallel_backend
from joblib import parallel_backend

with parallel_backend('multiprocessing'):
    print(5)
#    results = Parallel(n_jobs=4)(delayed(square)(i) for i in range(10))

In [ ]:
def decode(X, y, nperm= 2, n_jobs = 10, nbfold=5, nbfold_perm=2):
    assert len(X) == len(y), (X.shape, y.shape)
    good =  ~np.isnan(y) 
    yg = y[good]
    Xg = X[good]  # trial x channel x time   

    np.sum( np.isinf(yg) ), np.sum( np.isnan(yg) )
    gc.collect()
    scoring = make_scorer(scorer_spearman)
    
    #n_jobs = 1

    regression_type = 'Ridge'
     

    spoc = SPoC(n_components=5, log=True, reg='oas',                                                                                                        
                                       rank='full', n_jobs=n_jobs)
    #spoc = SPoC_orig(n_components=5, log=True, reg='oas',rank='full')                                                                                        

       # Regression for classic decoding                                                                                              
    if regression_type == 'Ridge':                                                                                     
        alphas = np.logspace(-5, 5, 12)                                                                                                                     
        est = make_pipeline(spoc, RidgeCV())                                                                                                                
        # Regressions for the B2B                                                                                                                           
#         G = make_pipeline(spoc,                                                                                                                             
#             RidgeCV(alphas=alphas, fit_intercept=False)) 
    cv = KFold(nbfold, shuffle=True)
    cvperm = KFold(nbfold_perm, shuffle=True)
    

    classic_dec_verbose = 1

    #%debug
    def sc(est,XX,YY): 
        print(XX.shape)
        return spearmanr(XX,YY)[0]
    yg_perm = np.random.permutation(yg)
    with mne.use_log_level('warning'):
        scores = cross_val_multiscore(est, Xg, y=yg, cv=cv,                                                                                    
               scoring=scoring, n_jobs=n_jobs,                                                                                             
               verbose=classic_dec_verbose) 

        scs = []
        for i in range(nperm):
            yg_perm = np.random.permutation(yg)
            scores_perm = cross_val_multiscore(est, Xg, y=yg_perm, cv=cvperm,                                                                                    
                   scoring=scoring, n_jobs=n_jobs,                                                                                             
                   verbose=0) 
            scs += [scores_perm.mean()]
        scores_perm = np.array(scs)

    
    
    return scores, scores_perm

In [ ]:
#scores, scores_perm = decode(X, y, nperm= 3, n_jobs = 10, nbfold=5)

In [ ]:
del tlim

In [ ]:
import time
nperm = 6
nbfold=5
nbfold_perm=2
n_jobs = 20
# gradient compensation = using subtracting ref_meg
res = []
for ph in phase2epochs_gen_info: 
    varnames_cur = phase2epochs_gen_info[ph]['varnames']
    varname2y = phase2dat[ph]['varname2y']
    
    if tlim_test is None:
        tlims = phase2epochs_gen_info[ph]['tlims']
    else:
        tlims = {'test':tlim_test}
    for tlim_name, tlim in tlims.items():
        X = phase2tlimn2ep[ph][tlim_name].copy().pick_types(meg=True, ref_meg=False)._data
        for vn in varnames_cur:
            t0 = time.time()
            print('-------- Starting ',ph, vn, X.shape, tlim)
            y = varname2y[vn]

            scores, scores_perm = decode(X, y, nperm= nperm, n_jobs = n_jobs, 
                                         nbfold=nbfold, nbfold_perm=nbfold_perm)
            t1 = time.time()
            td = t1-t0
            print(ph, vn, tlim, scores.mean(), scores_perm.mean(), td)
            res += [ (ph, vn, scores.mean(), scores_perm.mean(), X.shape, tlim, td)]        

In [ ]:
#res_not_ITI = res 
#res_ITI = res
#res_tot =  res_not_ITI + res_ITI

In [ ]:
dfdecres = pd.DataFrame(res_tot, columns=['phase','coln','spearman','spearman_perm', 'Xshape',
                                      'tlim', 'calc_dur'])
dfdecres['nperm'] =nperm
dfdecres['nbfold'] =nbfold
#dfdecres['tmin'] = tlim[0]
#dfdecres['tmax'] = tlim[1]
dfdecres

In [ ]:
dfdecres.to_csv(pjoin(p,f'{subj}_dec_res_test2.csv'))

In [ ]:
9 *7

# ICA

In [ ]:
import gc
gc.collect()

In [ ]:
raw.load_data(1)

In [ ]:
#https://mne.tools/stable/auto_tutorials/preprocessing/40_artifact_correction_ica.html#sphx-glr-auto-tutorials-preprocessing-40-artifact-correction-ica-py

In [ ]:
from mne.preprocessing import create_eog_epochs,create_ecg_epochs

In [ ]:
dat_pupilsz,time = raw[chn_pupil_size]

In [ ]:
raw.drop_channels(['EEG063-2800','EEG064-2800',chn_pupil_size])

In [ ]:
eog_epochs = create_eog_epochs(raw)  # decim possible

In [ ]:
eog_epochs.save(pjoin(p,f'eog-epo.fif'), overwrite=True)
eog_evoked = eog_epochs.average()
eog_evoked.apply_baseline(baseline=(None, -0.2))
eog_evoked.plot_joint(picks='meg');

In [ ]:
ecg_epochs = create_ecg_epochs(raw)   # decim possible
ecg_epochs.save(pjoin(p,f'ecg-epo.fif'))
ecg_evoked = ecg_epochs.average()
ecg_evoked.apply_baseline(baseline=(None, -0.2))
ecg_evoked.plot_joint(picks='meg');

In [ ]:
n_jobs = 40
filt_raw = raw.copy().filter(l_freq=1., h_freq=None, n_jobs=n_jobs)

In [ ]:
#ica.fit?

In [ ]:
from mne.preprocessing import ICA
ica = ICA(n_components=15, max_iter='auto', random_state=0)
ica.fit(filt_raw )
ica

In [ ]:
ica.save(pjoin(p,f'afterfilt-ica.fif'), overwrite=True)

In [ ]:
psd.plot(picks='meg',average=1)

In [ ]:
raw.plot(theme='dark');

In [ ]:
ica.plot_sources?

In [ ]:
%matplotlib qt
ica.plot_sources(raw, theme='dark', picks=[1,2,3,4]);

In [ ]:
# reject

In [ ]:
gc.collect()

In [ ]:
explained_var_ratio = ica.get_explained_variance_ratio(filt_raw)
for channel_type, ratio in explained_var_ratio.items():
    print(
        f'Fraction of {channel_type} variance explained by all components: '
        f'{ratio}'
    )

In [ ]:
ecg_indices

In [ ]:
ica.exclude = []
# find which ICs match the ECG pattern
ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method='correlation',
                                            threshold='auto')
print(ecg_indices)
if len(ecg_indices):
    ica.exclude = ecg_indices

    # barplot of ICA component "ECG match" scores
    ica.plot_scores(ecg_scores)

    # plot diagnostics
    ica.plot_properties(raw, picks=ecg_indices)

    # plot ICs applied to raw data, with ECG matches highlighted
    ica.plot_sources(raw, show_scrollbars=False)

    # plot ICs applied to the averaged ECG epochs, with ECG matches highlighted
    ica.plot_sources(ecg_evoked)

In [ ]:
eog_indices, eog_scores = ica.find_bads_eog(raw)
print(eog_indices)
if len(eog_indices):
    ica.exclude = eog_indices
    # barplot of ICA component "EOG match" scores
    ica.plot_scores(eog_scores)

    # plot diagnostics
    ica.plot_properties(raw, picks=eog_indices)

    # plot ICs applied to raw data, with EOG matches highlighted
    ica.plot_sources(raw, show_scrollbars=False)

    # plot ICs applied to the averaged EOG epochs, with EOG matches highlighted
    ica.plot_sources(eog_evoked)

In [ ]:
raw.load_data()
ica.plot_sources(raw, show_scrollbars=False)
#ica.plot_components()

# blinks
#ica.plot_overlay(raw, exclude=[0], picks='eeg')
# heartbeats
#ica.plot_overlay(raw, exclude=[1], picks='mag')

ica.plot_properties(raw, picks=[0, 1])

In [ ]:
reconst_raw = raw.copy()
ica.apply(reconst_raw)

raw.plot(order=artifact_picks, n_channels=len(artifact_picks),
         show_scrollbars=False)
reconst_raw.plot(order=artifact_picks, n_channels=len(artifact_picks),
                 show_scrollbars=False)
del reconst_raw

# Psd

In [ ]:
rawcrp = raw.copy()

rawcrp.crop(0,70)

rawcrp.crop?

r.times[-1], rawcrp.times[-1], raw.times[-1]

In [ ]:
rawcrp.compute_psd?

In [ ]:
ts = dfev[dfev['phase'] == 'REST'].head(1)['time'].values[0]

In [ ]:
te = dfev[dfev['phase'] == 'ITI'].tail(1)['time'].values[0]

In [ ]:
psd.plot?

In [ ]:
psd = raw.compute_psd(picks='meg', tmin=ts,tmax=te, fmax=200, n_jobs=20)

In [ ]:
%matplotlib inline
plt.figure(figsize=(12,4))
ax = plt.gca()
fig = psd.plot(picks='meg', axes= ax, average=True);
ax.set_xlim(0,200)

In [ ]:
# I am not sure I need notch filter -- I don't see clear peaks around 60 Hz in psd
#mne.filter.notch_filter(x, Fs, freqs, filter_length='auto', notch_widths=None, trans_bandwidth=1, method='fir', iir_params=None, mt_bandwidth=None, p_value=0.05, picks=None, n_jobs=None, copy=True, phase='zero', fir_window='hamming', fir_design='firwin', pad='reflect_limited', *, verbose=None)[source]


In [ ]:
%matplotlib qt
psd.plot_topo();

In [ ]:
psd.plot?

In [ ]:
raw.notch_filter()

# Maxwell

In [ ]:
sphere = mne.make_sphere_model(r0=(0., 0., 0.), head_radius=0.08)

mne.viz.plot_alignment(raw.info, subject='sample',
                       meg='helmet', bem=sphere, dig=True,
                       surfaces=['brain'])
#del raw, epochs

In [ ]:
#from here
# https://mne.tools/stable/auto_tutorials/inverse/85_brainstorm_phantom_ctf.html#sphx-glr-auto-tutorials-inverse-85-brainstorm-phantom-ctf-py
rawm = raw.copy()
rawm.apply_gradient_compensation(0)  # must un-do software compensation first
mf_kwargs = dict(origin=(0., 0., 0.), st_duration=10.)
rawm = mne.preprocessing.maxwell_filter(rawm, **mf_kwargs)
#rawm.plot()

In [ ]:
rawm.save(pjoin(p,'SSS-raw.fif'))

In [ ]:
len(inds)

In [ ]:
np.max(d)

In [ ]:
d[0,inds[[0,1000]]]

In [ ]:
d.shape

In [ ]:
raw.

In [ ]:
mne.find_events(raw, )

In [ ]:
%matplotlib qt
raw_nontriv.plot();

In [ ]:
raw_nontriv.ch_names